In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Installing required libraries

In [2]:
!pip install yfinance

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.2/63.2 kB 2.1 MB/s eta 0:00:00


In [3]:
import pandas as pd
import yfinance as yf
from datetime import datetime
print("done")

done


# Data Collection and Cleaning

Here's the code to collect real time data from the Yahoo finance API. The following command is used to customize the date for our dataset. We will use three months of data for analysis that we can find using this command - DateOffset.

We can do the similar thing for other time period as well. For example, we can analyse data for last 5 or 6 months in the same way. For now, let's analyse three months and see what can we conclude from historical data and it can help us make decison to make our next investment.

In [4]:
start_date = datetime.now() - pd.DateOffset(months =3) #DateOffset provides a flexible way to shift or adjust dates by specific intervals. Basically we shifted our date by 3 months.
end_date = datetime.now()

In [5]:
tickers  = ['AAPL','MSFT', 'NFLX', 'Goog', 'TSLA']

df_list = []

for ticker in tickers:
    data = yf.download(ticker, start=start_date, end=end_date)
#     print(data)
    df_list.append(data)
    
df = pd.concat(df_list, keys=tickers, names=['Ticker', 'Date'])
# print(print(df))
print(df.head())

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
                         Open        High         Low       Close   Adj Close  \
Ticker Date                                                                     
AAPL   2023-04-04  166.600006  166.839996  165.110001  165.630005  165.401230   
       2023-04-05  164.740005  165.050003  161.800003  163.759995  163.533798   
       2023-04-06  162.429993  164.960007  162.000000  164.660004  164.432556   
       2023-04-10  161.419998  162.029999  160.080002  162.029999  161.806183   
       2023-04-11  162.350006  162.360001  160.509995  160.800003  160.577896   

                     Volume  
Ticker Date                  
AAPL   2023-04-04  46278300

Now look at the following command. What it actually does is - it creates a new index. Initially date column was chosen as index by defualt that you can see above.

In [6]:
df= df.reset_index()
print(df.head())

  Ticker       Date        Open        High         Low       Close  \
0   AAPL 2023-04-04  166.600006  166.839996  165.110001  165.630005   
1   AAPL 2023-04-05  164.740005  165.050003  161.800003  163.759995   
2   AAPL 2023-04-06  162.429993  164.960007  162.000000  164.660004   
3   AAPL 2023-04-10  161.419998  162.029999  160.080002  162.029999   
4   AAPL 2023-04-11  162.350006  162.360001  160.509995  160.800003   

    Adj Close    Volume  
0  165.401230  46278300  
1  163.533798  51511700  
2  164.432556  45390100  
3  161.806183  47716900  
4  160.577896  47644200  


# Exploring Dataset using Charts

Time to look at the performance in the stock market of all the comapnies.

Here we import plotly express to work on our project. What's that?

**Plotly Express** is a high-level data visualization library built on top of Plotly, a powerful and flexible plotting library for Python. It aims to make it easy for users to quickly generate visualizations with minimal code and effort. It provides a set of functions that abstract away the complexity of the underlying Plotly library, allowing users to create visually appealing and interactive plots with just a few lines of code.

Rest you can chatGPT now xd!

In [7]:
import plotly.express as px
fig = px.line(df, x='Date', y='Close', color = 'Ticker',  title = 'Stock market performance for the last 3 months' )
fig.show()

Looks good!

By the way, Do you know what is faceting? And what does this ".rolling" function do in Pandas?

**Faceting** is a technique used in data visualization to display multiple subsets of data in a series of smaller, related plots within a larger plot or dashboard. It involves dividing a dataset into groups based on one or more variables and creating separate visualizations for each group.

The purpose of faceting is to provide a clearer and more detailed view of the data across different categories or dimensions. By presenting multiple plots side by side or in a grid layout, faceting allows for easy comparison and identification of patterns, trends, and variations within each subset of data.Faceting is particularly useful when dealing with high-dimensional datasets or when analyzing relationships between variables. It helps to reveal insights and relationships that may be obscured when viewing the data as a whole.

Now how to apply this python code? Let's look at that!

In [8]:
fig = px.area(df, x='Date', y = 'Close', color = 'Ticker', 
        facet_col = 'Ticker',
        labels = {'Date':'Date', 'Close':'Closing Price', 'Ticker':'Company'},
       title = 'Stock Prices for Apple, Microsoft, Netflix, Tesla and Google')
fig.show()

That's really cool! Prices are more clear now and infact we can visualize correlation among stocks as well. But we'll something else for it. Till then, let's move forward.

# Finding Moving Averages

In [9]:
# 10 days moving average

In [10]:
df['MA10'] = df.groupby('Ticker')['Close'].rolling(window=10).mean().reset_index(0, drop = True)

# 10 days MA is calculated using grouping 10 closing prices first and then finding avg of them.
# they are grouped using rolling funtion with window size 10 nd 
# then .mean function calculates mean of the same grouped data.

In [11]:
print(df['MA10'].head(22))

# you see it gives moving evg of every 10 stocks. 
# First 9 values are empty for the same reason.
# let's look at moving avg of 20 days as well.

0            NaN
1            NaN
2            NaN
3            NaN
4            NaN
5            NaN
6            NaN
7            NaN
8            NaN
9     163.945001
10    164.145001
11    164.434001
12    164.470001
13    164.800002
14    165.097002
15    165.463000
16    165.748001
17    166.195000
18    166.631000
19    166.837999
20    166.819998
21    166.733998
Name: MA10, dtype: float64


In [12]:
print(df)

    Ticker       Date        Open        High         Low       Close  \
0     AAPL 2023-04-04  166.600006  166.839996  165.110001  165.630005   
1     AAPL 2023-04-05  164.740005  165.050003  161.800003  163.759995   
2     AAPL 2023-04-06  162.429993  164.960007  162.000000  164.660004   
3     AAPL 2023-04-10  161.419998  162.029999  160.080002  162.029999   
4     AAPL 2023-04-11  162.350006  162.360001  160.509995  160.800003   
..     ...        ...         ...         ...         ...         ...   
305   TSLA 2023-06-27  243.240005  250.389999  240.850006  250.210007   
306   TSLA 2023-06-28  249.699997  259.880005  248.889999  256.239990   
307   TSLA 2023-06-29  258.029999  260.739990  253.610001  257.500000   
308   TSLA 2023-06-30  260.600006  264.450012  259.890015  261.769989   
309   TSLA 2023-07-03  276.489990  284.250000  275.109985  279.820007   

      Adj Close     Volume        MA10  
0    165.401230   46278300         NaN  
1    163.533798   51511700         NaN  


In [13]:
df['MA20'] = df.groupby('Ticker')['Close'].rolling(window=20).mean().reset_index(0, drop=True)

for ticker, group in df.groupby('Ticker'):
    print(f'Moving Averages for {ticker}')
    print(group[['MA10', 'MA20']])
    
    
    
# here group by is used for grouping the data according to the categories and applying some function to it. 
# For example, here we applied mean function.
# .rolling function in pandas allows you to apply a specified function to a sliding window of data over a defined window size.
# Once you have applied the rolling function, you can chain it with various aggregation functions such as mean(), sum(), min(), 
# max(), etc., to compute the desired rolling calculation.

Moving Averages for AAPL
          MA10        MA20
0          NaN         NaN
1          NaN         NaN
2          NaN         NaN
3          NaN         NaN
4          NaN         NaN
..         ...         ...
57  185.416998  182.584500
58  186.010999  183.182000
59  186.574998  183.799000
60  187.370999  184.493000
61  188.125000  185.068501

[62 rows x 2 columns]
Moving Averages for Goog
           MA10      MA20
186         NaN       NaN
187         NaN       NaN
188         NaN       NaN
189         NaN       NaN
190         NaN       NaN
..          ...       ...
243  122.875999  123.6870
244  122.541000  123.5090
245  122.104000  123.3410
246  121.622000  123.1710
247  121.272000  122.9375

[62 rows x 2 columns]
Moving Averages for MSFT
           MA10        MA20
62          NaN         NaN
63          NaN         NaN
64          NaN         NaN
65          NaN         NaN
66          NaN         NaN
..          ...         ...
119  337.156998  333.802499
120  337.312997  33

# Visualizing Moving Averages

In [14]:
# I guess it's complex to understand what's happening just by looking at numbers. Let's visualize it

for ticker, group in df.groupby('Ticker'):
    fig = px.line(group, x ='Date', y=['Close', 'MA10', 'MA20'], title = f"{ticker} moving averages")
    fig.show()

# **Decision Making**

But what do we understand from above graphs? The output shows four separate graphs for each company. 

* When the MA10 crosses above the MA20, it is considered a bullish signal indicating that the stock price will continue to rise. 
* Conversely, when the MA10 crosses below the MA20, it is a bearish signal that the stock price will continue falling.

There's another metric we can use to analyze share prices and that's : **Volatility**.

Volatility is a key metric in financial markets as it reflects the level of uncertainty or risk associated with an investment. Higher volatility indicates larger price swings, indicating higher potential gains or losses. Conversely, lower volatility suggests relatively stable price movements.

There are two main types of volatility:

* Historical Volatility: This type of volatility is based on past price data. It measures the standard deviation of the stock's or market's price over a specific period, usually calculated on a daily or annual basis. **Historical volatility provides *insights* into the stock's price behavior and can be used to assess risk.**

* Implied Volatility: Implied volatility represents the market's expectation of future volatility. It is derived from options prices, specifically the premiums investors are willing to pay for options contracts. Implied volatility reflects investors' sentiment and expectations regarding potential future price movements.

For our analysis we'll use *historical volatility*. 

Here we are using ".pct_change" function. In pandas, **pct_change()** is a function that calculates the percentage change between consecutive elements in a Series or DataFrame column. It is used to compute the relative change in values over a specified period.

In [15]:
df['Volatility'] = df.groupby('Ticker')['Close'].pct_change().rolling(window=10).std().reset_index(0, drop=True)
fig = px.line(df, x='Date', y='Volatility', 
              color='Ticker', 
              title='Volatility of All Companies')
fig.show()

In [16]:
# print(df.Volatility)
# print(df)
px.line(df.Volatility,)

Imp note : A volatility of 0.01 indicates that the stock's price tends to experience relatively small or modest fluctuations. This suggests that the stock's price is relatively stable over the specified period, and the risk associated with sudden large price movements is relatively low. Investors who prefer less risk or are looking for more stable investments may find stocks with lower volatility more appealing.

Volatility can vary across different time periods. A stock may exhibit low volatility over a short period but higher volatility over a longer timeframe. Therefore, it's recommended to consider volatility in relation to the specific time period and compare it to other stocks or market benchmarks to gain a more comprehensive understanding of the stock's risk profile.

# Correlation among Stocks

Now comes one of the most important metric and that's correlation between stocks. How does change in stock price of one share can affect the price of other stock?

Let's start with Apple and Microsoft. We'll do the same with other stocks as well, and check if one price affects the other.

In [17]:
apple = df.loc[df['Ticker']=='AAPL', ['Date', 'Close']].rename(columns = {'Close':'AAPL'})

microsoft = df.loc[df['Ticker'] == 'MSFT', ['Date', 'Close']].rename(columns={'Close': 'MSFT'})

print(apple)
print(microsoft)

         Date        AAPL
0  2023-04-04  165.630005
1  2023-04-05  163.759995
2  2023-04-06  164.660004
3  2023-04-10  162.029999
4  2023-04-11  160.800003
..        ...         ...
57 2023-06-27  188.059998
58 2023-06-28  189.250000
59 2023-06-29  189.589996
60 2023-06-30  193.970001
61 2023-07-03  192.460007

[62 rows x 2 columns]
          Date        MSFT
62  2023-04-04  287.179993
63  2023-04-05  284.339996
64  2023-04-06  291.600006
65  2023-04-10  289.390015
66  2023-04-11  282.829987
..         ...         ...
119 2023-06-27  334.570007
120 2023-06-28  335.850006
121 2023-06-29  335.049988
122 2023-06-30  340.540009
123 2023-07-03  337.989990

[62 rows x 2 columns]


In [18]:
df_corr=pd.merge(apple, microsoft, on = 'Date')
print(df_corr)

         Date        AAPL        MSFT
0  2023-04-04  165.630005  287.179993
1  2023-04-05  163.759995  284.339996
2  2023-04-06  164.660004  291.600006
3  2023-04-10  162.029999  289.390015
4  2023-04-11  160.800003  282.829987
..        ...         ...         ...
57 2023-06-27  188.059998  334.570007
58 2023-06-28  189.250000  335.850006
59 2023-06-29  189.589996  335.049988
60 2023-06-30  193.970001  340.540009
61 2023-07-03  192.460007  337.989990

[62 rows x 3 columns]


Now it looks good. Let's look at visualization to understand correlation between them.

In [19]:
fig = px.scatter(df_corr, x='AAPL', y='MSFT', 
                 trendline='ols', 
                 title='Correlation between Apple and Microsoft')
fig.show()

Ohkay, what's that now?

Strong correlation between stock prices means that there is a significant statistical relationship or tendency for the prices of two or more stocks to move together in a similar direction. In other words, when one stock's price increases or decreases, there is a higher likelihood that the other stock(s) will also experience a similar movement. 

**There is a strong linear relationship between the stock prices of Apple and Microsoft, which means that when the stock price of Apple increases, the stock price of Microsoft also tends to increase.**

When two stocks have a strong positive correlation, it means that they tend to move in the same direction. For example, if stock A and stock B have a strong positive correlation, when the price of stock A goes up, there is a high probability that the price of stock B will also go up. Conversely, if the price of stock A goes down, the price of stock B is likely to decrease as well.

On the other hand, a strong negative correlation indicates an inverse relationship between stock prices. If two stocks have a strong negative correlation, when the price of one stock goes up, the price of the other stock tends to go down, and vice versa.



**what's there for investors?**

For investors, this positive correlation may indicate an opportunity to diversify their portfolio by investing in both companies, as both stocks may offer similar potential returns and risks.

So these insights and analysis will help us make informed decisions. Share your thoughts and feedback to improve this in the comments. Thanks to Aman Kharwal for guidance. 